# DIMLP Fidex

In [1]:
import pandas as pd
import pathlib as pl

import kagglehub

from random import randint
from omnixai_community.data.tabular import Tabular
from omnixai_community.visualization.dashboard import Dashboard
from omnixai_community.explainers.tabular.auto import TabularExplainer
from mlxplain.explainers.tabular.specific.dimlpfidex import DimlpBTModel

# ------------------------


In [20]:
root_dir = pl.Path("../datasets/HeartDataset/")
dataset = pd.read_csv(root_dir.joinpath("heart.csv"))

In [13]:

cp = pd.get_dummies(
    dataset["cp"], prefix="cp", prefix_sep="_", columns=["cp"], dtype="int8"
)
dataset = pd.concat([dataset.iloc[:, :2], cp, dataset.iloc[:, 3:]], axis=1)
dataset = dataset.rename(
    columns={
        "cp_0": "cp_typical",
        "cp_1": "cp_atypical",
        "cp_2": "cp_nonanginal ",
        "cp_3": "cp_asymptomatic",
    }
)

output = pd.get_dummies(
    dataset["output"], prefix="risk", prefix_sep="_", columns=["output"], dtype="int8"
)

dataset = pd.concat([dataset.iloc[:, :-1], output], axis=1)
dataset = dataset.rename(columns={"risk_0": "risk_no", "risk_1": "risk_yes"})

dataset

,age,sex,cp_typical,cp_atypical,cp_nonanginal,cp_asymptomatic,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,risk_no,risk_yes
0,63,1,0,0,0,1,145,233,1,0,150,0,2.3,0,0,1,0,1
1,37,1,0,0,1,0,130,250,0,1,187,0,3.5,0,0,2,0,1
2,41,0,0,1,0,0,130,204,0,0,172,0,1.4,2,0,2,0,1
3,56,1,0,1,0,0,120,236,0,1,178,0,0.8,2,0,2,0,1
4,57,0,1,0,0,0,120,354,0,1,163,1,0.6,2,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,1,0,0,0,140,241,0,1,123,1,0.2,1,0,3,1,0
299,45,1,0,0,0,1,110,264,0,1,132,0,1.2,1,0,3,1,0
300,68,1,1,0,0,0,144,193,1,1,141,0,3.4,1,2,3,1,0
301,57,1,1,0,0,0,130,131,0,1,115,1,1.2,1,1,3,1,0


In [15]:
dataset = dataset.sample(frac=1)
split = int(dataset.shape[0] * 0.8)

features = dataset.columns

nb_classes = 2
nb_features = len(features) - nb_classes

train_dataset = Tabular(
    data=dataset.iloc[:split,:]
)

test_dataset = Tabular(
    data=dataset.iloc[split:,:]
)

features_filename = "attributes.txt"

with open(root_dir.joinpath(features_filename), "w") as file:
    for feature in features:
        file.write(f"{feature}\n")

In [19]:
features

Index(['age', 'sex', 'cp_typical', 'cp_atypical', 'cp_nonanginal ',
       'cp_asymptomatic', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'risk_no', 'risk_yes'],
      dtype='object')

In [16]:
# WARNING: verbose_console can't be used when used in notebooks for some reason

model = DimlpBTModel(
    root_dir,
    train_dataset,
    test_dataset,
    nb_features,
    nb_classes,
    attributes_file=features_filename,
)

model.train()

0

In [21]:
max_iterations = 10
min_covering = 2
max_failed_attempts = 15
min_fidelity = 1.0
lowest_fidelity_allowed = 0.9
use_minimal_version = True

explainer = TabularExplainer(
    explainers=["fidex", "fidexGloRules"],
    data=train_dataset,
    model=model,
    mode="classification",
    params={
        "fidex": {
            "max_iterations": max_iterations,
            "attributes_file": features_filename,
            "min_covering": min_covering,
            "max_failed_attempts": max_failed_attempts,
            "min_fidelity": min_fidelity,
            "lowest_min_fidelity": lowest_fidelity_allowed,
        },
        "fidexGloRules": {
            "heuristic": 1,
            "with_fidexGlo": True,
            "fidexGlo": {
                "attributes_file": features_filename,
                "with_minimal_version": use_minimal_version,
                "max_iterations": max_iterations,
                "min_covering": min_covering,
                "covering_strategy": True,
                "max_failed_attempts": max_failed_attempts,
                "min_fidelity": min_fidelity,
                "lowest_min_fidelity": lowest_fidelity_allowed,
            },
            "attributes_file": features_filename,
            "nb_threads": 4,
            "with_minimal_version": use_minimal_version,
            "max_iterations": max_iterations,
            "min_covering": min_covering,
        },
    },
)

# predict with random sample
sample_to_test = test_dataset[randint(0, len(test_dataset)-1)]

local_explainations = explainer.explain(X=sample_to_test)
global_explainations = explainer.explain_global()

dashboard = Dashboard(
    instances=sample_to_test,
    local_explanations=local_explainations,
    global_explanations=global_explainations,
    class_names=features[-2:],
)
dashboard.show()